In [1]:
import random
import numpy as np
import gym
import matplotlib
import matplotlib.pyplot as plt

from insoco.Environment import Environment
from insoco.Plotting import plotQ

In [2]:
env = Environment("CliffWalking")

env.reset()
env.render()
for a in [1,1,2,2,1,1]:
    env.step(a)

Setting default max_steps per episode: 100000
CliffWalking--1ag Discrete Action Space with Discrete(4)
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T



In [3]:
def policy_random():
    return env.action_space.sample()

def policy_epsilon(state, epsilon, q_values):
    return 0

In [54]:
class Q_function():  
    # stores the Q_function Q(s,a) = "future reward" as a dictionnary:
    # Q(s) = [ reward per action ]
    # either as a table (or function approximation, next notebook)    
    # stores the number of actions of the agent needed to add a new output 

    def __init__(self, env):       
        self.nactions = env.action_space.n
        self.f = {}
        
    def predict(self, s, a=None):        
        return 0
    
    def update(self, s, a, y):
        pass


In [51]:
def Q_learning(env, learning_rate=0.3, discount_factor=0.9, num_episodes=5000, num_episodes_end=4000):
    epsilon_ini = 1    
    np.random.seed() 
    
    env.history = {"episode_rew":[], "episode_steps":[]}
    
    Q =  Q_function(env)
        
    env.close()
    return Q 

In [52]:
params = {"learning_rate":0.1, "discount_factor":0.8, "num_episodes":5000, "num_episodes_end":4000}
q_func = Q_learning(env, **params)
plt.plot(env.history["episode_rew"])

In [48]:
def SARSA_learning(env, learning_rate=0.3, discount_factor=0.9, num_episodes=5000, num_episodes_end=4000):
    epsilon_ini = 1    
    np.random.seed() 
    env.history = {"episode_rew":[], "episode_steps":[]}
    
    Q =  Q_function(env)
    
    sum_rew_epi = 0
        
    env.close()
    return Q 

In [49]:
# Careful with the discount factor for SARSA it may not converge if < 1
# Reason being: as SARSA being on-policy and risc averse, if we act greedy with respect to reward it does not converge

params = {"learning_rate":0.1, "discount_factor":0.999, "num_episodes":5000, "num_episodes_end":4000}
q_func = SARSA_learning(env, **params)
plt.plot(env.history["episode_rew"])